In [ ]:
## Goal

Let's follow the steps in https://pytorch.org/tutorials/advanced/super_resolution_with_onnxruntime.html and see what happens!

## Prerequsite

Install pytorch 1.8.1 as https://pytorch.org/ guides
```
pip3 install torch==1.8.1+cpu torchvision==0.9.1+cpu torchaudio==0.8.1 -f https://download.pytorch.org/whl/torch_stable.html
```

Install onnx and onnxruntime
```
pip install onnx==1.9.0 onnxruntime==1.7.0
```


In [2]:
# Some standard imports
import io
import numpy as np

In [3]:
# Some torch imports
from torch import nn
import torch.utils.model_zoo as model_zoo
import torch.onnx
import torch.nn as nn
import torch.nn.init as init

In [4]:
# Super Resolution model definition in PyTorch

class SuperResolutionNet(nn.Module):
    def __init__(self, upscale_factor, inplace=False):
        super(SuperResolutionNet, self).__init__()

        self.relu = nn.ReLU(inplace=inplace)
        self.conv1 = nn.Conv2d(1, 64, (5, 5), (1, 1), (2, 2))
        self.conv2 = nn.Conv2d(64, 64, (3, 3), (1, 1), (1, 1))
        self.conv3 = nn.Conv2d(64, 32, (3, 3), (1, 1), (1, 1))
        self.conv4 = nn.Conv2d(32, upscale_factor ** 2, (3, 3), (1, 1), (1, 1))
        self.pixel_shuffle = nn.PixelShuffle(upscale_factor)

        self._initialize_weights()

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = self.relu(self.conv3(x))
        x = self.pixel_shuffle(self.conv4(x))
        return x

    def _initialize_weights(self):
        init.orthogonal_(self.conv1.weight, init.calculate_gain('relu'))
        init.orthogonal_(self.conv2.weight, init.calculate_gain('relu'))
        init.orthogonal_(self.conv3.weight, init.calculate_gain('relu'))
        init.orthogonal_(self.conv4.weight)

# Create the super-resolution model by using the above model definition.
torch_model = SuperResolutionNet(upscale_factor=3)

In [5]:
# set the model to inference mode
torch_model.eval()

SuperResolutionNet(
  (relu): ReLU()
  (conv1): Conv2d(1, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(64, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(32, 9, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pixel_shuffle): PixelShuffle(upscale_factor=3)
)

In [10]:
batch_size = 1    # just a random number

# Input to the model
x = torch.randn(batch_size, 1, 224, 224, requires_grad=True)
torch_out = torch_model(x)

In [17]:
# Export the model
# 
# https://pytorch.org/docs/stable/onnx.html#tracing-vs-scripting refers to this API as 'trace' API
torch.onnx.export(torch_model,               # model being run
                  x,                         # model input (or a tuple for multiple inputs)
                  "super_resolution.A.onnx",   # where to save the model (can be a file or file-like object)
                  export_params=True,        # store the trained parameter weights inside the model file
                  opset_version=10,          # the ONNX version to export the model to
                  do_constant_folding=True,  # whether to execute constant folding for optimization
                  verbose=True,
                  input_names = ['input'],   # the model's input names
                  output_names = ['output'], # the model's output names
                  dynamic_axes={'input' : {0 : 'batch_size'},    # variable length axes
                                'output' : {0 : 'batch_size'}})

graph(%input : Float(*, 1, 224, 224, strides=[50176, 50176, 224, 1], requires_grad=1, device=cpu),
      %conv1.weight : Float(64, 1, 5, 5, strides=[25, 25, 5, 1], requires_grad=1, device=cpu),
      %conv1.bias : Float(64, strides=[1], requires_grad=1, device=cpu),
      %conv2.weight : Float(64, 64, 3, 3, strides=[576, 9, 3, 1], requires_grad=1, device=cpu),
      %conv2.bias : Float(64, strides=[1], requires_grad=1, device=cpu),
      %conv3.weight : Float(32, 64, 3, 3, strides=[576, 9, 3, 1], requires_grad=1, device=cpu),
      %conv3.bias : Float(32, strides=[1], requires_grad=1, device=cpu),
      %conv4.weight : Float(9, 32, 3, 3, strides=[288, 9, 3, 1], requires_grad=1, device=cpu),
      %conv4.bias : Float(9, strides=[1], requires_grad=1, device=cpu)):
  %9 : Float(*, 64, 224, 224, strides=[3211264, 50176, 224, 1], requires_grad=1, device=cpu) = onnx::Conv[dilations=[1, 1], group=1, kernel_shape=[5, 5], pads=[2, 2, 2, 2], strides=[1, 1]](%input, %conv1.weight, %conv1.bias) # 

In [16]:
torch.onnx.export(torch_model,               # model being run
                  x,                         # model input (or a tuple for multiple inputs)
                  "super_resolution.B.onnx",   # where to save the model (can be a file or file-like object)
                  verbose=True)
# Q. How to write a multi output model?

graph(%input.1 : Float(1, 1, 224, 224, strides=[50176, 50176, 224, 1], requires_grad=1, device=cpu),
      %conv1.weight : Float(64, 1, 5, 5, strides=[25, 25, 5, 1], requires_grad=1, device=cpu),
      %conv1.bias : Float(64, strides=[1], requires_grad=1, device=cpu),
      %conv2.weight : Float(64, 64, 3, 3, strides=[576, 9, 3, 1], requires_grad=1, device=cpu),
      %conv2.bias : Float(64, strides=[1], requires_grad=1, device=cpu),
      %conv3.weight : Float(32, 64, 3, 3, strides=[576, 9, 3, 1], requires_grad=1, device=cpu),
      %conv3.bias : Float(32, strides=[1], requires_grad=1, device=cpu),
      %conv4.weight : Float(9, 32, 3, 3, strides=[288, 9, 3, 1], requires_grad=1, device=cpu),
      %conv4.bias : Float(9, strides=[1], requires_grad=1, device=cpu)):
  %9 : Float(1, 64, 224, 224, strides=[3211264, 50176, 224, 1], requires_grad=1, device=cpu) = onnx::Conv[dilations=[1, 1], group=1, kernel_shape=[5, 5], pads=[2, 2, 2, 2], strides=[1, 1]](%input.1, %conv1.weight, %conv1.bias

In [25]:
# How to write a model with multiple outputs?
class MOModel(nn.Module):
    def __init__(self):
        super(MOModel, self).__init__()

        self.relu = nn.ReLU(inplace=False)

    def forward(self, x):
        lhs = self.relu(x)
        rhs = self.relu(x)
        return lhs, rhs

mo_model = MOModel()

In [26]:
torch.onnx.export(mo_model, x, "MO.B.onnx", verbose=True)

graph(%input : Float(1, 1, 224, 224, strides=[50176, 50176, 224, 1], requires_grad=1, device=cpu)):
  %1 : Float(1, 1, 224, 224, strides=[50176, 50176, 224, 1], requires_grad=1, device=cpu) = onnx::Relu(%input) # /home/parjong/opt/python3/lib/python3.8/site-packages/torch/nn/functional.py:1206:0
  %2 : Float(1, 1, 224, 224, strides=[50176, 50176, 224, 1], requires_grad=1, device=cpu) = onnx::Relu(%input) # /home/parjong/opt/python3/lib/python3.8/site-packages/torch/nn/functional.py:1206:0
  return (%1, %2)

